In [ ]:
import numpy as np
import statsmodels.api as sm

rng = np.random.default_rng()

In [3]:
def sample_sparse_linear(n_samples=100, n_features=20, n_nonzero=5, noise_std=0.1):
    """
    Generate data from a sparse linear model with Gaussian noise.

    Parameters
    ----------
    n_samples : int
        Number of samples
    n_features : int
        Number of features
    n_nonzero : int
        Number of nonzero coefficients
    noise_std : float
        Standard deviation of Gaussian noise
    random_state : int
        Random seed

    Returns
    -------
    X : array, shape (n_samples, n_features)
        Feature matrix
    y : array, shape (n_samples,)
        Target vector
    beta : array, shape (n_features,)
        True coefficients
    """
    # Generate random feature matrix
    X = rng.standard_normal((n_samples, n_features))

    # Generate sparse coefficients
    beta = np.zeros(n_features)
    nonzero_idx = rng.choice(n_features, size=n_nonzero, replace=False)
    beta[nonzero_idx] = rng.standard_normal(n_nonzero)

    # Generate target with noise
    y = X @ beta + noise_std * rng.standard_normal(n_samples)

    return X, y, beta

In [4]:
X, y, beta_true = sample_sparse_linear()
# Adding interecept column to X
X = np.hstack([np.ones((X.shape[0], 1)), X])

In [5]:
def loss_func(y, y_hat):
    return np.sum(np.square(y - y_hat))


def loss_func_deriv(X, beta, y):
    return -2 * X.T @ (y - X @ beta)


def check_obj_stop(f_old, f_new, eps):
    return np.abs(f_new - f_old) < eps * np.abs(f_old)


def check_params_stop(params_old, params_new, eps):
    return np.linalg.norm(params_new - params_old) < eps * np.linalg.norm(params_old)

In [ ]:
beta = np.zeros(X.shape[1])
eps_params = 1e-5
eps_obj = 1e-5

while True:
    del_f = loss_func_deriv(X, beta, y)
    print(np.linalg.norm(del_f))
    p = -del_f

    if np.linalg.norm(del_f) < 1e-5:
        break

    def find_step_size(alpha=1, c=0.5, tau=0.5):
        # Backtracking line search
        # https://en.wikipedia.org/wiki/Backtracking_line_search
        m = loss_func_deriv(X, beta, y).T @ p
        t = -c * m
        while loss_func(X @ beta, y) - loss_func(X @ (beta + alpha * p), y) < alpha * t:
            alpha = tau * alpha
        return alpha

    beta_new = beta + find_step_size() * p
    if check_obj_stop(
        loss_func(y, X @ beta), loss_func(y, X @ beta_new), eps_obj
    ) or check_params_stop(beta, beta_new, eps_params):
        break

    beta = beta_new

676.7973046204713
342.2761547552429
126.67538572635736
67.68384942477226
46.68496333554782
22.516704972150098
14.57815409776227
11.298869967455445
6.077860016132026
5.504937192383445
2.64817067588246
1.7913813795609295
1.3929655982258204
0.7715146377771303
0.6757503573761724
0.33715337753919444
0.23181681081578018
0.1741392800699846
0.09981944389793353
0.08384535525339581
0.04345585701086835
0.0411161399521352


In [6]:
beta[1:].round(2)

array([ 0.02,  0.01,  0.03, -2.08, -0.01,  0.02, -0.  ,  0.03,  0.  ,
        0.  ,  0.  , -0.  ,  0.01, -0.16, -0.  ,  0.  , -1.6 , -1.36,
       -1.48,  0.  ])

In [7]:
beta_true.round(2)

array([ 0.  ,  0.  ,  0.  , -2.08,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  , -0.16,  0.  ,  0.  , -1.6 , -1.35,
       -1.48,  0.  ])

In [6]:
def prox_op(x, lam):
    return np.sign(x) * np.maximum((np.abs(x) - lam), 0)

In [9]:
beta = np.zeros(X.shape[1])
lam = 0.01
eps_params = 1e-5
eps_obj = 1e-5

while True:
    del_f = loss_func_deriv(X, beta, y)
    print(np.linalg.norm(del_f))
    p = -del_f

    def find_step_size(alpha=1, c=0.5, tau=0.5):
        # Backtracking line search
        # https://en.wikipedia.org/wiki/Backtracking_line_search
        m = loss_func_deriv(X, beta, y).T @ p
        t = -c * m
        while loss_func(X @ beta, y) - loss_func(X @ (beta + alpha * p), y) < alpha * t:
            alpha = tau * alpha
        return alpha

    beta_new = prox_op(beta + find_step_size() * p, lam)
    if check_obj_stop(
        loss_func(y, X @ beta), loss_func(y, X @ beta_new), eps_obj
    ) or check_params_stop(beta, beta_new, eps_params):
        break

    beta = beta_new

676.7973046204713
350.0573086153344
125.88164431583853
70.03564780167251
40.5362254477239
19.18461300213021
11.73979837729861
11.196462590622499
9.403728947997001
9.012624930720934
8.39339981092738
8.233875299098766
8.146457683724242
8.113722452910492
8.09728659271841
8.089816512147406
8.086074222198322
8.084228766546477


In [10]:
beta[1:].round(2)

array([ 0.  , -0.  ,  0.01, -2.06,  0.  ,  0.  , -0.  ,  0.  ,  0.  ,
        0.  , -0.  ,  0.  , -0.  , -0.15,  0.  ,  0.  , -1.59, -1.35,
       -1.47, -0.  ])

In [11]:
beta_true.round(2)

array([ 0.  ,  0.  ,  0.  , -2.08,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  , -0.16,  0.  ,  0.  , -1.6 , -1.35,
       -1.48,  0.  ])

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

In [32]:
def sample_sparse_gamma(n_samples=100, n_features=20, n_nonzero=5, noise_scale=0.1):
    """
    Generate data from a sparse linear model with Gamma noise.

    Parameters
    ----------
    n_samples : int
        Number of samples
    n_features : int
        Number of features
    n_nonzero : int
        Number of nonzero coefficients
    noise_scale : float
        Scale parameter for Gamma noise

    Returns
    -------
    X : array, shape (n_samples, n_features)
        Feature matrix
    y : array, shape (n_samples,)
        Target vector
    beta : array, shape (n_features,)
        True coefficients
    """
    # Generate random feature matrix
    # X = rng.standard_normal((n_samples, n_features))
    X = rng.uniform(0.1, 0.5, (n_samples, n_features))

    # Generate sparse coefficients
    beta = np.zeros(n_features)
    nonzero_idx = rng.choice(n_features, size=n_nonzero, replace=False)
    beta[nonzero_idx] = rng.standard_normal(n_nonzero)

    # Generate target with gamma noise
    eta = X @ beta
    mu = np.exp(eta)  # Log link
    shape = 1 / noise_scale
    scale = mu / shape  # Set scale so mean = mu
    y = rng.gamma(shape, scale)

    return X, y, beta

In [131]:
X, y, beta_true = sample_sparse_gamma(n_samples=100000, n_features=10, noise_scale=1)
# Adding interecept column to X
X = np.hstack([np.ones((X.shape[0], 1)), X])

In [148]:
def loss_func(y_hat, y):
    # y_hat = jnp.exp(y_hat)
    # return 2 * jnp.sum(-jnp.log(y / y_hat) + (y - y_hat) / y_hat)
    return 2 * jnp.sum(y_hat + y * jnp.exp(-y_hat))


def gen_grad(x, del_g, prox_op, t):
    return (x - prox_op(x - t * del_g)) / t


loss_func_deriv = jax.grad(lambda beta: loss_func(X @ beta, y))
loss_func_hess = jax.hessian(lambda beta: loss_func(X @ beta, y))

In [ ]:
beta = np.hstack([[1], np.zeros(X.shape[1] - 1)])
lam = 0.01
eps_params = 1e-5
eps_obj = 1e-5

while True:
    grad = loss_func_deriv(beta)
    hess = loss_func_hess(beta)
    del_g = jnp.linalg.solve(hess, grad)

    # When n_features to n_samples ratio starts to get large better results with tau=0.8 and c=0.75
    # Converge speed seems to depend greatly on selections of line search parameters compared to that ratio
    # TODO: Find better step size selection algorithm
    def find_step_size(t=1, c=0.25, tau=0.8, max_iter=100):
        # Backtracking line search
        # https://en.wikipedia.org/wiki/Backtracking_line_search
        current_loss = loss_func(X @ beta, y)
        for _ in range(max_iter):
            G = gen_grad(beta, del_g, lambda x: prox_op(x, lam), t)
            new_loss = loss_func(X @ (beta - t * G), y)
            if jnp.isnan(new_loss) or (
                new_loss
                > current_loss - t * del_g.T @ G + t / 2 * jnp.linalg.norm(G) ** 2
            ):
                t = tau * t
            else:
                break
        print(f"Step size: {t}")
        return t

    t = find_step_size()
    beta_new = beta - t * gen_grad(beta, del_g, lambda x: prox_op(x, lam), t)
    print(f"Param diff: {np.linalg.norm(beta_new - beta)}")
    if check_obj_stop(
        loss_func(X @ beta, y), loss_func(X @ beta_new, y), eps_obj
    ) and check_params_stop(beta, beta_new, eps_params):
        break

    beta = beta_new

Step size: 0.8
Param diff: 4.135533960814055
Step size: 1
Param diff: 1.1878580670121504
Step size: 1
Param diff: 1.0813510405434958
Step size: 1
Param diff: 0.2843124918808631
Step size: 3.484491437270418e-05
Param diff: 0.026525890804703668
Step size: 1
Param diff: 0.021149545894374375
Step size: 2.7875931498163346e-05
Param diff: 0.02658519363112942
Step size: 1
Param diff: 0.02677324559020136
Step size: 2.7875931498163346e-05
Param diff: 0.02658854918318828
Step size: 1
Param diff: 0.02658305349929286
Step size: 2.7875931498163346e-05
Param diff: 0.026588532891142564
Step size: 1
Param diff: 0.026588149585396438
Step size: 2.7875931498163346e-05
Param diff: 0.026588410800489858
Step size: 1
Param diff: 0.026589176383429194
Step size: 2.7875931498163346e-05
Param diff: 0.026588489857099863
Step size: 1
Param diff: 0.02658847172300747
Step size: 2.7875931498163346e-05
Param diff: 0.02658849341610533
Step size: 1
Param diff: 0.02658769732153098
Step size: 2.7875931498163346e-05
Param 

KeyboardInterrupt: 

In [171]:
beta[1:].round(2)

array([-1.  ,  0.4 ,  0.01, -0.65,  0.  ,  0.59, -0.04, -0.13, -0.  ,
       -0.  ])

In [172]:
beta_true.round(2)

array([-1.05,  0.45,  0.  , -0.67,  0.  ,  0.57,  0.  , -0.14,  0.  ,
        0.  ])